In [88]:
# aggregate all political queries
import os
import pandas as pd
for root, dir, files in os.walk('queries'):
    df = pd.DataFrame()
    for file in files:
        if file.endswith('.csv'):
            filepath = os.path.join(root,file)
            temp_df = pd.read_csv(filepath)
            df = pd.concat([df, temp_df])

df

,Unnamed: 0,raw-group,summarized-query
0,0,"Donald Trump, Joe Biden, United States preside...",Trump Biden Debate
1,1,"Pension, Democratic Party, Pension fund, Bob C...",Pension Democratic Fund Casey Biden
2,2,"Ukraine, Joe Biden, Russia, CNN",Ukraine Biden Russia CNN
3,3,"Republican National Convention, Republican Par...",RNC Republican Milwaukee
4,4,"Debate, Devon, Cornwall, General election, Lib...",Devon Cornwall Debate Election
...,...,...,...
5,5,"Democratic Party, Primary election, New York's...",NY 1st Dem Primary CNN
6,6,"Republican National Convention, Republican Par...",RNC Milwaukee GOP
7,7,"Debate, Devon, Cornwall, General election, Lib...",Devon Cornwall Election Debate
8,8,"California, Gavin Newsom, United States Congre...",California Newsom Congress Democrats


# G-TAB Dataframe for Queries

This notebook uses G-TAB to gather Google Trends information for queries. 
Updated and added documentation 1/24/23 to get data for FAccT paper. This notebook uses the time period 6/1-8/1 and saves information about search interest per day per query, with information for all queries saved as a CSV.

In [39]:
import core
import time
import json
import csv
import os
import random

Create the anchorbank for US and between 6/1/22-8/1/22. This takes a while to run and you have to be careful not to get an error for too many requests.

In [2]:
# USE anchorbank.py instead for create_anchorbank(). Just do this:
my_path = "june_current_us"
t = core.GTAB(dir_path=my_path)

Set active anchorbank to the one just made.

In [3]:

t.set_active_gtab("google_anchorbank_geo=US_timeframe=2024-06-28 2024-07-12.tsv")

Read in list of queries to gather data for. 

In [13]:
len(queryList)

15

The output for one query is a dataframe, run the following for an example:

To keep track of information by query, we will add a column 'query' to the dataframe and fill in the relevant query for all of its rows. Run the following for an example.

Now we will gather data for all the queries we have. We need to include pauses to avoid too many requests errors (though they may still happen, in which case we try to switch computers). Everything is written to a CSV file.

In [100]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[
logging.FileHandler("gtab_log.log"),
])

def main(queryList, type):
    """Gets calibrated google trends information per query using the 
    appropriate anchorbank that was set active above."""
    with open(f'{type}-validQueries.csv', 'w') as f:
        w = csv.writer(f, delimiter='\n')
        w.writerow(["query"])
    with open(f'{type}-badQueries.csv','w') as f:
        writer = csv.DictWriter(f, ["query"])
        writer.writeheader()
    with open('processed.csv','w') as f:
        writer = csv.DictWriter(f,["query"])
        writer.writeheader()
    while True:
        processed = []
        if os.path.isfile('processed.csv'):
            with open('processed.csv','r') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    processed.append(row['query'])
        if len(queryList) == len(processed):
            break
        for i, query in enumerate(queryList):
            if query not in processed:
                try:
                    logging.info(f"processing {i}th query: {query}")
                    with open('processed.csv','a') as f:
                            writer = csv.DictWriter(f,["query"])
                            writer.writerow({'query':query})
                    q = t.new_query(query)
                    try:
                        count=0
                        q['query'] = q['max_ratio'].apply(lambda x: query)
                        csvInfo = q.to_csv()
                        csvLines = csvInfo.split('\n')
                        with open(f'{type}-gtabData-test.csv', 'a') as f: #append info to csv file (not sure if this is the best way to do this but it works)
                            w = csv.writer(f, delimiter='\n')
                            for line in csvLines[1:]:
                                if line != "": #sometimes there were empty lines, so exclude those 
                                    w.writerow([line])
                        with open(f'{type}-validQueries.csv', 'a') as f:
                            w = csv.writer(f, delimiter='\n')
                            w.writerow([query])
                        time.sleep(random.randint(5,15))
                    except Exception as e:
                        logging.info(e)
                        with open(f'{type}-badQueries.csv','a') as f:
                            writer = csv.DictWriter(f, ["query"])
                            writer.writerow({'query': query})
                except Exception as e: #'bad query' exception (not enough google trends data)
                    count+=1
                    if count==5: # if we run into a request error 5 times in a row
                        time.sleep(300)
                    logging.info(e)
                    time.sleep(random.randint(10,20))
                    pass


In [73]:
q_file = "/Users/enistudent/Desktop/anchorbank/gtrends-trending@06-28-11.csv"

with open(q_file) as f:
    r = csv.reader(f)
    next(r)
    queryList = [''.join(row[1].split(',')[:2]).replace(',','') if len(row[1].split(',')) > 3 else row[1].replace(',','') for row in r]
    
queryList

['Supreme Court of the United States Chevron U.S.A.',
 'Thomas Massie Rhonda Massie',
 'California United States Congress',
 'Joe Biden Donald Trump',
 'Infinera Nike',
 'Hamas Joe Biden',
 'Yield Inflation',
 'Supreme Court of the United States Social media Joe Biden',
 'Democratic Party George Latimer',
 'Iowa Abortion law Supreme Court of the United States',
 'Michigan Government budget Gretchen Whitmer',
 'Jill Biden Joe Biden',
 'Republican National Convention Republican Party Milwaukee',
 'Merrick Garland Republican Party',
 'Robert F. Kennedy Jr. North Carolina']

In [98]:
queryList = df['summarized-query'].to_list()
print(len(queryList), queryList[:5])

202 ['Trump Biden Debate', 'Pension Democratic Fund Casey Biden', 'Ukraine Biden Russia CNN', 'RNC Republican Milwaukee', 'Devon Cornwall Debate Election']


In [101]:
main(queryList, 'allPoliticalQueriesSoFar')

HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22Trump+Biden+Debate%22%2C+%22time%22%3A+%222024-06-28+2024-07-12%22%2C+%22geo%22%3A+%22US%22%7D%5D%2C+%22category%22%3A+%220%22%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))


KeyboardInterrupt: 